In [1]:
# import torch
# import random
# import os
# import numpy as np
# import networkx as nx
# import torch.nn as nn
# import torch.nn.functional as F
# import torch as th
# import matplotlib
# from collections import OrderedDict, defaultdict
# from networkx.algorithms.flow import shortest_augmenting_path
# from dgl.nn.pytorch import GraphConv
# from itertools import chain, islice, combinations
# from networkx.algorithms.approximation.clique import maximum_independent_set as mis
# from time import time
# from networkx.algorithms.approximation.maxcut import one_exchange
# from itertools import permutations
# import matplotlib.pyplot as plt
# import dgl
# import pickle
from commons import *

Separate code base from heurestics
# Utils code

In [25]:
TORCH_DEVICE = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
TORCH_DTYPE = torch.float32


def partition_weight(adj, s):
    """
    Calculates the sum of weights of edges that are in different partitions.

    :param adj: Adjacency matrix of the graph.
    :param s: List indicating the partition of each edge (0 or 1).
    :return: Sum of weights of edges in different partitions.
    """
    s = np.array(s)
    partition_matrix = np.not_equal.outer(s, s).astype(int)
    weight = (adj * partition_matrix).sum() / 2
    return weight

import torch

def partition_weight2(adj, s):
    """
    Calculates the sum of weights of edges that are in different partitions.

    :param adj: Adjacency matrix of the graph as a PyTorch tensor.
    :param s: Tensor indicating the partition of each node (0 or 1).
    :return: Sum of weights of edges in different partitions.
    """
    # Ensure s is a tensor
    # s = torch.tensor(s, dtype=torch.float32)

    # Compute outer difference to create partition matrix
    s = s.unsqueeze(0)  # Convert s to a row vector
    t = s.t()           # Transpose s to a column vector
    partition_matrix = (s != t).float()  # Compute outer product and convert boolean to float

    # Calculate the weight of edges between different partitions
    weight = (adj * partition_matrix).sum() / 2

    return weight

def calculateAllCut(q_torch, s):
    '''

    :param q_torch: The adjacent matrix of the graph
    :param s: The binary output from the neural network. s will be in form of [[prob1, prob2, ..., prob n], ...]
    :return: The calculated cut loss value
    '''
    if len(s) > 0:
        totalCuts = len(s[0])
        CutValue = 0
        for i in range(totalCuts):
            CutValue += partition_weight2(q_torch, s[:,i])
        return CutValue/2
    return 0

def hyperParameters(n = 100, d = 3, p = None, graph_type = 'reg', number_epochs = int(1e5),
                    learning_rate = 1e-4, PROB_THRESHOLD = 0.5, tol = 1e-4, patience = 100):
    dim_embedding = int(np.sqrt(4096))    # e.g. 10
    hidden_dim = int(dim_embedding/2)

    return n, d, p, graph_type, number_epochs, learning_rate, PROB_THRESHOLD, tol, patience, dim_embedding, hidden_dim
def FIndAC(graph):
    max_degree = max(dict(graph.degree()).values())
    A_initial = max_degree + 1  # A is set to be one more than the maximum degree
    C_initial = max_degree / 2  # C is set to half the maximum degree

    return A_initial, C_initial



# Neural Network Model

# Training Neural network

In [55]:


def run_gnn_training2(dataset, net, optimizer, number_epochs, tol, patience, loss_func, dim_embedding, total_classes=3, save_directory=None, torch_dtype = TORCH_DTYPE, torch_device = TORCH_DEVICE, labels=None):
    """
    Train a GCN model with early stopping.
    """
    # loss for a whole epoch
    prev_loss = float('inf')  # Set initial loss to infinity for comparison
    prev_cummulative_loss = float('inf')
    cummulativeCount = 0
    count = 0  # Patience counter
    best_loss = float('inf')  # Initialize best loss to infinity
    best_model_state = None  # Placeholder for the best model state
    loss_list = []
    epochList = []
    cumulative_loss = 0

    t_gnn_start = time()

    # contains information regarding all terminal nodes for the dataset
    terminal_configs = {}
    epochCount = 0
    criterion = nn.BCELoss()
    A = nn.Parameter(torch.tensor([65.0]))
    C = nn.Parameter(torch.tensor([32.5]))

    embed = nn.Embedding(80, dim_embedding)
    embed = embed.type(torch_dtype).to(torch_device)
    inputs = embed.weight

    for epoch in range(number_epochs):

        cumulative_loss = 0.0  # Reset cumulative loss for each epoch

        for key, (dgl_graph, adjacency_matrix,graph, terminals) in dataset.items():
            epochCount +=1


            # Ensure model is in training mode
            net.train()

            # Pass the graph and the input features to the model
            logits = net(dgl_graph, inputs)

            # Compute the loss
            # loss = loss_func(criterion, logits, labels, terminals[0], terminals[1])

            loss = loss_func( logits, dgl_graph)


            # Backpropagation
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()

            # Update cumulative loss
            cumulative_loss += loss.item()



            # # Check for early stopping
            if epoch > 0 and (cumulative_loss > prev_loss or abs(prev_loss - cumulative_loss) <= tol):
                count += 1
                if count >= patience: # play around with patience value, try lower one
                    print(f'Stopping early at epoch {epoch}')
                    break
            else:
                count = 0  # Reset patience counter if loss decreases

            # Update best model
            if cumulative_loss < best_loss:
                best_loss = cumulative_loss
                best_model_state = net.state_dict()  # Save the best model state

        loss_list.append(loss)

        # # Early stopping break from the outer loop
        # if count >= patience:
        #     count=0

        prev_loss = cumulative_loss  # Update previous loss

        if epoch % 100 == 0:  # Adjust printing frequency as needed
            print(f'Epoch: {epoch}, Cumulative Loss: {cumulative_loss}')

            if save_directory != None:
                checkpoint = {
                    'epoch': epoch,
                    'model': net.state_dict(),
                    'optimizer': optimizer.state_dict(),
                    'lossList':loss_list,
                    'inputs':inputs}
                torch.save(checkpoint, './epoch'+str(epoch)+'loss'+str(cumulative_loss)+ save_directory)

            if (prev_cummulative_loss == cummulativeCount):
                cummulativeCount+=1

                if cummulativeCount > 4:
                    break
            else:
                prev_cummulative_loss = cumulative_loss


    t_gnn = time() - t_gnn_start

    # Load the best model state
    if best_model_state is not None:
        net.load_state_dict(best_model_state)

    print(f'GNN training took {round(t_gnn, 3)} seconds.')
    print(f'Best cumulative loss: {best_loss}')
    loss = loss_func(logits, adjacency_matrix)
    if save_directory != None:
        checkpoint = {
            'epoch': epoch,
            'model': net.state_dict(),
            'optimizer': optimizer.state_dict(),
            'lossList':loss_list,
            'inputs':inputs}
        torch.save(checkpoint, './final_'+save_directory)

    return net, best_loss, epoch, inputs, loss_list

## HyperParameters initialization and related functions

In [4]:



def printCombo(orig):
    # Original dictionary
    input_dict = orig

    # Generate all permutations of the dictionary values
    value_permutations = list(permutations(input_dict.values()))

    # Create a list of dictionaries from the permutations
    permuted_dicts = [{key: value for key, value in zip(input_dict.keys(), perm)} for perm in value_permutations]

    return permuted_dicts

def GetOptimalNetValue(net, dgl_graph, inp, q_torch, terminal_dict):
    net.eval()
    best_loss = float('inf')

    if (dgl_graph.number_of_nodes() < 30):
        inp = torch.ones((dgl_graph.number_of_nodes(), 30))

    # find all potential combination of terminal nodes with respective indices

    perm_items = printCombo(terminal_dict)
    for i in perm_items:
        probs = net(dgl_graph, inp, i)
        binary_partitions = (probs >= 0.5).float()
        cut_value_item = calculateAllCut(q_torch, binary_partitions)
        if cut_value_item < best_loss:
            best_loss = cut_value_item
    return best_loss



# Hamiltonian loss function

In [5]:
def terminal_independence_penalty(s, terminal_nodes):
    """
    Calculate a penalty that enforces each terminal node to be in a distinct partition.
    :param s: A probability matrix of size |V| x |K| where s[i][j] is the probability of vertex i being in partition j.
    :param terminal_nodes: A list of indices for terminal nodes.
    :return: The penalty term.
    """
    penalty = 0
    num_terminals = len(terminal_nodes)
    # Compare each pair of terminal nodes
    for i in range(num_terminals):
        for j in range(i + 1, num_terminals):
            # Calculate the dot product of the probability vectors for the two terminals
            dot_product = torch.dot(s[terminal_nodes[i]], s[terminal_nodes[j]])
            # Penalize the similarity in their partition assignments (dot product should be close to 0)
            penalty += dot_product
    return penalty

In [6]:
def calculate_HA_vectorized(s):
    """
    Vectorized calculation of HA.
    :param s: A binary matrix of size |V| x |K| where s[i][j] is 1 if vertex i is in partition j.
    :return: The HA value.
    """
    # HA = ∑v∈V(∑k∈K(sv,k)−1)^2
    HA = torch.sum((torch.sum(s, axis=1) - 1) ** 2)
    return HA

def calculate_HC_min_cut_intra_inter(s, adjacency_matrix):
    """
    Vectorized calculation of HC to minimize cut size.
    :param s: A probability matrix of size |V| x |K| where s[i][j] is the probability of vertex i being in partition j.
    :param adjacency_matrix: A matrix representing the graph where the value at [i][j] is the weight of the edge between i and j.
    :return: The HC value focusing on minimizing edge weights between partitions.
    """
    HC = 0
    K = s.shape[1]
    for k in range(K):
        for l in range(k + 1, K):
            partition_k = s[:, k].unsqueeze(1) * s[:, k].unsqueeze(0)  # Probability node pair both in partition k
            partition_l = s[:, l].unsqueeze(1) * s[:, l].unsqueeze(0)  # Probability node pair both in partition l
            # Edges between partitions k and l
            inter_partition_edges = adjacency_matrix * (partition_k + partition_l)
            HC += torch.sum(inter_partition_edges)

    return HC

def calculate_HC_min_cut_intra_inter2(s, adjacency_matrix):
    """
    Vectorized calculation of HC to minimize cut size.
    :param s: A probability matrix of size |V| x |K| where s[i][j] is the probability of vertex i being in partition j.
    :param adjacency_matrix: A matrix representing the graph where the value at [i][j] is the weight of the edge between i and j.
    :return: The HC value focusing on minimizing edge weights between partitions.
    """
    HC = 0
    K = s.shape[1]
    for k in range(K):
        for l in range(k + 1, K):
            partition_k = s[:, k].unsqueeze(1) * s[:, k].unsqueeze(0)  # Probability node pair both in partition k
            partition_l = s[:, l].unsqueeze(1) * s[:, l].unsqueeze(0)  # Probability node pair both in partition l
            # Edges between partitions k and l
            inter_partition_edges = adjacency_matrix * (partition_k + partition_l)
            HC += torch.sum(inter_partition_edges)

    return HC

def calculate_HC_min_cut_new(s, adjacency_matrix):
    """
    Differentiable calculation of HC for minimizing edge weights between different partitions.
    :param s: A probability matrix of size |V| x |K| where s[i][j] is the probability of vertex i being in partition j.
    :param adjacency_matrix: A matrix representing the graph where the value at [i][j] is the weight of the edge between i and j.
    :return: The HC value, focusing on minimizing edge weights between partitions.
    """
    K = s.shape[1]
    V = s.shape[0]

    # Create a full partition matrix indicating the likelihood of each node pair being in the same partition
    partition_matrix = torch.matmul(s, s.T)

    # Calculate the complement matrix, which indicates the likelihood of node pairs being in different partitions
    complement_matrix = 1 - partition_matrix

    # Apply adjacency matrix to only consider actual edges and their weights
    inter_partition_edges = adjacency_matrix * complement_matrix

    # Summing up all contributions for edges between different partitions
    HC = torch.sum(inter_partition_edges)

    return HC

def calculate_HC_vectorized_old(s, adjacency_matrix):
    """
    Vectorized calculation of HC.
    :param s: A binary matrix of size |V| x |K|.
    :param adjacency_matrix: A matrix representing the graph where the value at [i][j] is the weight of the edge between i and j.
    :return: The HC value.
    """
    # HC = ∑(u,v)∈E(1−∑k∈K(su,k*sv,k))*adjacency_matrix[u,v]
    K = s.shape[1]
    # Outer product to find pairs of vertices in the same partition and then weight by the adjacency matrix
    prod = adjacency_matrix * (1 - s @ s.T)
    HC = torch.sum(prod)
    return HC
import torch

def min_cut_loss(s, adjacency_matrix):
    """
    Compute a differentiable min-cut loss for a graph given node partition probabilities.

    :param s: A probability matrix of size |V| x |K| where s[i][j] is the probability of vertex i being in partition j.
    :param adjacency_matrix: A matrix representing the graph where the value at [i][j] is the weight of the edge between i and j.
    :return: The expected min-cut value, computed as a differentiable loss.
    """
    V = s.size(0)  # Number of nodes
    K = s.size(1)  # Number of partitions

    # Ensure the partition matrix s sums to 1 over partitions
    s = torch.softmax(s, dim=1)

    # Compute the expected weight of edges within each partition
    intra_partition_cut = torch.zeros((K, K), dtype=torch.float32)
    for k in range(K):
        for l in range(k + 1, K):
            # Probability that a node pair (i, j) is split between partitions k and l
            partition_k = s[:, k].unsqueeze(1)  # Shape: V x 1
            partition_l = s[:, l].unsqueeze(0)  # Shape: 1 x V

            # Compute the expected weight of the cut edges between partitions k and l
            cut_weight = adjacency_matrix * (partition_k @ partition_l)
            intra_partition_cut[k, l] = torch.sum(cut_weight)

    # Sum up all contributions to get the total expected min-cut value
    total_cut_weight = torch.sum(intra_partition_cut)

    return total_cut_weight

import torch

# def min_cut_loss(s, adjacency_matrix):
#     """
#     Compute a differentiable min-cut loss for a graph given node partition probabilities.
#
#     :param s: A probability matrix of size |V| x |K| where s[i][j] is the probability of vertex i being in partition j.
#     :param adjacency_matrix: A matrix representing the graph where the value at [i][j] is the weight of the edge between i and j.
#     :return: The expected min-cut value, computed as a differentiable loss.
#     """
#     V = s.size(0)  # Number of nodes
#     K = s.size(1)  # Number of partitions
#
#     # Ensure the partition matrix s sums to 1 over partitions
#     # s = torch.softmax(s, dim=1)
#
#     # Compute the expected weight of cut edges between each pair of partitions
#     total_cut_weight = 0
#     for k in range(K):
#         for l in range(k + 1, K):
#             # Probability that a node pair (i, j) is split between partitions k and l
#             partition_k = s[:, k].unsqueeze(1)  # Shape: V x 1
#             partition_l = s[:, l].unsqueeze(0)  # Shape: 1 x V
#
#             # Compute the expected weight of the cut edges between partitions k and l
#             cut_weight = adjacency_matrix * (partition_k @ partition_l)
#             total_cut_weight += torch.sum(cut_weight)
#
#     return total_cut_weight


def calculate_HC_vectorized(s, adjacency_matrix):
    """
    Vectorized calculation of HC for soft partitioning.
    :param s: A probability matrix of size |V| x |K| where s[i][j] is the probability of vertex i being in partition j.
    :param adjacency_matrix: A matrix representing the graph where the value at [i][j] is the weight of the edge between i and j.
    :return: The HC value.
    """
    # Initialize HC to 0
    HC = 0

    # Iterate over each partition to calculate its contribution to HC
    for k in range(s.shape[1]):
        # Compute the probability matrix for partition k
        partition_prob_matrix = s[:, k].unsqueeze(1) * s[:, k].unsqueeze(0)

        # Compute the contribution to HC for partition k
        HC_k =adjacency_matrix * (1 - partition_prob_matrix)
        # Sum up the contributions for partition k
        HC += torch.sum(HC_k, dim=(0, 1))

    # Since we've summed up the partition contributions twice (due to symmetry), divide by 2
    HC = HC / 2

    return HC




In [7]:
s = torch.Tensor([[0,1,0],[0,1,0],[0,0,1]])
# print(calculate_HA_vectorized(s))
# print(calculate_HA_vectorized(torch.Tensor([[0,0.9,0.9],[0.9,0.9,0],[0,0,0.9]])))
terminal_loss = torch.abs(s[0] - s[1]-s[2])
# print(terminal_loss)
# print(10 * (1 - terminal_loss))
# print(torch.sum(10 * (1 - terminal_loss)))
print(torch.abs(s[0] - s[1]))
print(torch.abs(s[0] - s[2]))
print(torch.abs(s[2] - s[1]))

print(torch.sum(10 * (1-torch.abs(s[0] - s[1]))))
print(torch.sum(10 * (1-torch.abs(s[0] - s[2]))))
print(torch.sum(10 * (1-torch.abs(s[2] - s[1]))))
print(terminal_independence_penalty(s, [0,1,2]))

tensor([0., 0., 0.])
tensor([0., 1., 1.])
tensor([0., 1., 1.])
tensor(30.)
tensor(10.)
tensor(10.)
tensor(1.)


In [19]:
def train1(modelName):
    n, d, p, graph_type, number_epochs, learning_rate, PROB_THRESHOLD, tol, patience, dim_embedding, hidden_dim = hyperParameters(learning_rate=0.001, n=4096,patience=20)

    # Establish pytorch GNN + optimizer
    opt_params = {'lr': learning_rate}
    gnn_hypers = {
        'dim_embedding': dim_embedding,
        'hidden_dim': hidden_dim,
        'dropout': 0.0,
        'number_classes': 3,
        'prob_threshold': PROB_THRESHOLD,
        'number_epochs': number_epochs,
        'tolerance': tol,
        'patience': patience,
        'nodes':n
    }
    datasetItem = open_file('./testData/prepareDS.pkl')
    # print(datasetItem)
    # datasetItem_all = {}
    # for key, (dgl_graph, adjacency_matrix,graph) in datasetItem.items():
    #     A, C = FIndAC(graph)
    #     datasetItem_all[key] = [dgl_graph, adjacency_matrix, graph, A, C]

    # print(len(datasetItem), datasetItem[0][3])
    # datasetItem_2 = {}
    # datasetItem_2[0]=datasetItem[1]
    # print(datasetItem_2)

    net, embed, optimizer = get_gnn(n, gnn_hypers, opt_params, TORCH_DEVICE, TORCH_DTYPE)


    # print(datasetItem[1][2].nodes)
    # # Visualize graph
    # pos = nx.kamada_kawai_layout(datasetItem[1][2])
    # nx.draw(datasetItem[1][2], pos, with_labels=True, node_color=[[.7, .7, .7]])
    # cut_value, (part_1, part_2) = nx.minimum_cut(datasetItem_2[0][2], datasetItem_2[0][3][1], datasetItem_2[0][3][0], flow_func=shortest_augmenting_path)

    # print(cut_value, len(part_1), len(part_2))

    # resultList = []
    # all_indexes = sorted(part_1.union(part_2))
    # # Check membership for each index and append the appropriate pair to the result list
    # for index in all_indexes:
    #     if index in part_1:
    #         resultList.append([1, 0])
    #     elif index in part_2:
    #         resultList.append([0, 1])

    #
    trained_net, bestLost, epoch, inp, lossList= run_gnn_training2(
        datasetItem, net, optimizer, int(500),
        gnn_hypers['tolerance'], gnn_hypers['patience'], loss_terminal,gnn_hypers['dim_embedding'], gnn_hypers['number_classes'], modelName,  TORCH_DTYPE,  TORCH_DEVICE)

    return trained_net, bestLost, epoch, inp, lossList


### Neural Network Training, Setting A to 0

In [58]:
def Loss(s, adjacency_matrix,  A=1, C=1):
    HA = calculate_HA_vectorized(s)
    HC = calculate_HC_vectorized(s, adjacency_matrix)
    # HC = calculate_HC_min_cut_new(s, adjacency_matrix)
    # HC = calculate_HC_min_cut_intra_inter(s, adjacency_matrix)
    return A * HA + C * HC


def loss_terminal(s, adjacency_matrix,  A=10000, C=1, penalty=10000):
    loss = Loss(s, adjacency_matrix, A, C)
    loss += penalty* terminal_independence_penalty(s, [0,1,2])
    return loss

trained_net, bestLost, epoch, inp, lossList = train1('_80wayCut_LossOrig.pth')


Epoch: 0, Cumulative Loss: 1587806.7822265625


KeyboardInterrupt: 

In [21]:
# def corrected_mysteryLoss(s, adjacency_matrix):
#     loss = 0
#     K = s.shape[1]
#     for k in range(K):
#         for l in range(k + 1, K):
#             partition_k = s[:, k].unsqueeze(1)
#             partition_l = s[:, l].unsqueeze(0)
#
#             # partition_k = s[:, k].unsqueeze(1) * s[:, k].unsqueeze(0)
#             # partition_l = s[:, l].unsqueeze(1) * s[:, l].unsqueeze(0)
#
#             inter_partition_edges = adjacency_matrix * (partition_k + partition_l)
#             loss += torch.sum(inter_partition_edges)
#
#     return loss

def soft_min_cut_loss(s, adjacency_matrix):
    """
    Calculate a soft min-cut loss that maintains differentiability by penalizing
    the sum of squared differences from binary values (0 or 1).
    """
    s = torch.softmax(s, dim=1)  # Ensure that s is a proper probability distribution
    V, K = s.shape

    min_cut_loss = 0
    for k in range(K):
        for l in range(k + 1, K):
            # Use probabilities directly for nodes being in partitions k and l
            # partition_k = s[:, k].unsqueeze(1)
            # partition_l = s[:, l].unsqueeze(0)

            partition_k = s[:, k].unsqueeze(1) * s[:, k].unsqueeze(0)
            # partition_l = s[:, l].unsqueeze(1) * s[:, l].unsqueeze(0)
            partition_l = s[:, l].unsqueeze(0)
            # Edge weights between partitions
            inter_partition_edges = adjacency_matrix * (partition_k * partition_l)
            min_cut_loss += torch.sum(inter_partition_edges)

    # Regularization to encourage probabilities close to 0 or 1
    regularization = torch.sum((s * (1 - s)))

    return min_cut_loss + 0.01 * regularization  # Adjust regularization weight as necessary

def Loss(s, adjacency_matrix,  A=1, C=1):
    HA = calculate_HA_vectorized(s)
    # HC = calculate_HC_vectorized(s, adjacency_matrix)
    # HC = calculate_HC_min_cut_new(s, adjacency_matrix)
    HC = soft_min_cut_loss(s, adjacency_matrix)
    # HC = min_cut_loss(s, adjacency_matrix)
    return A * HA + C * HC


def loss_terminal(s, adjacency_matrix,  A=100, C=100, penalty=100000):
    loss = Loss(s, adjacency_matrix, A, C)
    loss += penalty* terminal_independence_penalty(s, [0,1,2])
    return loss

trained_net, bestLost, epoch, inp, lossList = train1('_80wayCut_Lossinter.pth')

Epoch: 0, Cumulative Loss: 14220161.884765625
Epoch: 100, Cumulative Loss: 6690870.7734375
Epoch: 200, Cumulative Loss: 6689155.9052734375
Epoch: 300, Cumulative Loss: 6688431.19140625
Epoch: 400, Cumulative Loss: 6688069.7470703125
Epoch: 500, Cumulative Loss: 6687530.076171875
Epoch: 600, Cumulative Loss: 6687087.41015625
Epoch: 700, Cumulative Loss: 6687105.1103515625
Epoch: 800, Cumulative Loss: 6687147.55859375
Epoch: 900, Cumulative Loss: 6687304.62109375
GNN training took 338.853 seconds.
Best cumulative loss: 28800.201171875


In [51]:
# def corrected_mysteryLoss(s, adjacency_matrix):
#     loss = 0
#     K = s.shape[1]
#     for k in range(K):
#         for l in range(k + 1, K):
#             partition_k = s[:, k].unsqueeze(1)
#             partition_l = s[:, l].unsqueeze(0)
#
#             # partition_k = s[:, k].unsqueeze(1) * s[:, k].unsqueeze(0)
#             # partition_l = s[:, l].unsqueeze(1) * s[:, l].unsqueeze(0)
#
#             inter_partition_edges = adjacency_matrix * (partition_k + partition_l)
#             loss += torch.sum(inter_partition_edges)
#
#     return loss

def soft_min_cut_loss(s, adjacency_matrix):
    """
    Calculate a soft min-cut loss that maintains differentiability by penalizing
    the sum of squared differences from binary values (0 or 1).
    """
    s = torch.softmax(s, dim=1)  # Ensure that s is a proper probability distribution
    V, K = s.shape

    min_cut_loss = 0
    for k in range(K):
        for l in range(k + 1, K):
            # Use probabilities directly for nodes being in partitions k and l
            # partition_k = s[:, k].unsqueeze(1)
            # partition_l = s[:, l].unsqueeze(0)

            partition_k = s[:, k].unsqueeze(1) * s[:, k].unsqueeze(0)
            partition_l = s[:, l].unsqueeze(1) * s[:, l].unsqueeze(0)
            # partition_l = s[:, l].unsqueeze(0)
            # Edge weights between partitions
            inter_partition_edges = adjacency_matrix * (partition_k @ partition_l)
            min_cut_loss += torch.sum(inter_partition_edges)

    # Regularization to encourage probabilities close to 0 or 1
    regularization = torch.sum((s * (1 - s)))

    return min_cut_loss + 0.01 * regularization  # Adjust regularization weight as necessary

# def min_cut_loss(s, adjacency_matrix):
#     """
#     Calculate the min-cut loss.
#     :param s: Partition matrix.
#     :param adjacency_matrix: Adjacency matrix of the graph.
#     :return: Min-cut loss value.
#     """
#     edge_loss = 0
#     for i in range(adjacency_matrix.shape[0]):
#         for j in range(adjacency_matrix.shape[1]):
#             if adjacency_matrix[i, j] > 0:
#                 edge_loss += adjacency_matrix[i, j] * (s[i] - s[j]).abs().sum()
#     return edge_loss

def calculate_multiclass_cut_value( s, A):

    num_partitions = s.shape[1]
    n = s.shape[0]

    # Initialize the cut value
    cut_value = torch.tensor(0.0, dtype=torch.float32)

    # Calculate contributions for all pairs of different partitions
    for i in range(num_partitions):
        for j in range(i + 1, num_partitions):
            # Probabilities of nodes being in partitions i and j
            p_i = s[:, i]
            p_j = s[:, j]

            # Outer products to calculate probabilities of being in different partitions
            P_outer = torch.ger(p_i, p_j)
            Q_outer = torch.ger(p_j, p_i)

            # Expected cut-value matrix for partitions i and j
            expected_cut_matrix = A * (P_outer + Q_outer)

            # Add to total cut value
            cut_value += torch.sum(torch.triu(expected_cut_matrix, 1))

    return cut_value
# def HC_(s, adjacency_matrix):
#     # Ensure probabilities is a row vector
#     probabilities = s.view(1, -1)
#
#     # Use probabilities directly for core and halo assignments
#     core_a = probabilities  # Probabilities of being in partition A
#     core_b = 1 - probabilities  # Probabilities of being in partition B
#     # Calculate H_B using probabilistic assignments
#     expected_connections = torch.matmul(core_a.T, core_b) + torch.matmul(core_b.T, core_a)
#     discrepancy_matrix = adjacency_matrix - expected_connections
#     H_B = torch.sum(discrepancy_matrix ** 2)
#
#     # Calculate H_C as the sum of squared probabilities in each partition's halo
#     H_C = torch.sum(core_b) ** 2 + torch.sum(core_a) ** 2
#
#     return HC

def partition_weight_2(adj, s):
    """
    Calculates the sum of weights of edges that are in different partitions.

    :param adj: Adjacency matrix of the graph as a PyTorch tensor.
    :param s: Tensor indicating the partition of each node (0 or 1).
    :return: Sum of weights of edges in different partitions.
    """
    # Ensure s is a tensor
    # s = torch.tensor(s, dtype=torch.float32)

    # Compute outer difference to create partition matrix
    s = s.unsqueeze(0) * s.unsqueeze(1)  # Convert s to a row vector
    t = s.t()           # Transpose s to a column vector
    partition_matrix = (s != t).float()  # Compute outer product and convert boolean to float

    # Calculate the weight of edges between different partitions
    weight = (adj * partition_matrix).sum() / 2

    return weight

def calculateAllCut_2(q_torch, s):
    '''

    :param q_torch: The adjacent matrix of the graph
    :param s: The binary output from the neural network. s will be in form of [[prob1, prob2, ..., prob n], ...]
    :return: The calculated cut loss value
    '''
    if len(s) > 0:
        totalCuts = len(s[0])
        CutValue = 0
        for i in range(totalCuts):
            CutValue += partition_weight_2(q_torch, s[:,i])
        return CutValue/2
    return 0

def partition_weight_3(adj, s):
    """
    Calculates the expected sum of weights of edges between nodes in different partitions,
    using probabilities of node partition assignments.

    :param adj: Adjacency matrix of the graph as a PyTorch tensor.
    :param s: Tensor indicating the probability of each node being in a given partition.
    :return: Expected sum of weights of edges between different partitions.
    """
    # s should be a vector of probabilities that each node is in the partition
    # Compute the probability matrix where each element (i, j) is the probability
    # that node i and node j are in different partitions
    s_outer = s.unsqueeze(0)
    partition_matrix = torch.abs(s_outer)  # Absolute difference gives the probability of being in different partitions

    # Calculate the expected weight of edges between different partitions
    weight = (adj * partition_matrix).sum() / 2

    return weight

def calculateAllCut_3(q_torch, s):
    """
    Calculates the total expected cut loss across all partitions.

    :param q_torch: The adjacency matrix of the graph.
    :param s: The output probabilities from the neural network for each partition.
    :return: The total calculated cut loss value.
    """
    if len(s) > 0:
        totalCuts = s.shape[1]  # Assuming s is of shape [num_nodes, num_partitions]
        CutValue = 0
        # Iterate over all pairs of partitions
        for i in range(totalCuts):
            for j in range(i + 1, totalCuts):
                # Calculate the expected cut weight between partition i and partition j
                CutValue += partition_weight_3(q_torch, s[:, i] - s[:, j])
        return CutValue
    return 0



def calculate_HC_vectorized(s, adjacency_matrix):
    """
    Vectorized calculation of HC.
    :param s: A binary matrix of size |V| x |K|.
    :param adjacency_matrix: A matrix representing the graph where the value at [i][j] is the weight of the edge between i and j.
    :return: The HC value.
    """
    # HC = ∑(u,v)∈E(1−∑k∈K(su,k*sv,k))*adjacency_matrix[u,v]
    K = s.shape[1]
    # Outer product to find pairs of vertices in the same partition and then weight by the adjacency matrix
    prod = adjacency_matrix * (1 - s @ s.T)
    HC = torch.sum(prod)
    return HC

def Loss(s, adjacency_matrix,  A=1, C=1):
    # HA = calculate_HA_vectorized(s)
    # HC = calculate_HC_vectorized(s, adjacency_matrix)
    # HC = calculate_HC_min_cut_new(s, adjacency_matrix)
    # HC = calculate_HC_vectorized(s, adjacency_matrix)
    HC = soft_min_cut_loss(s, adjacency_matrix)
    # HC = min_cut_loss(s, adjacency_matrix)
    return  (C * HC)


def loss_terminal(s, adjacency_matrix,  A=0, C=1, penalty=10000):
    loss = Loss(s, adjacency_matrix, A, C)
    # loss += (penalty* terminal_independence_penalty(s, [0,1,2]))
    return loss

class GCNSoftmax(nn.Module):
    def __init__(self, in_feats, hidden_size, num_classes, dropout, device):
        super(GCNSoftmax, self).__init__()
        self.dropout_frac = dropout
        self.conv1 = GraphConv(in_feats, hidden_size).to(device)
        self.conv2 = GraphConv(hidden_size, num_classes).to(device)

    def forward(self, g, inputs):
        # Basic forward pass
        h = self.conv1(g, inputs)
        h = F.relu(h)
        h = F.dropout(h, p=self.dropout_frac, training=self.training)
        h = self.conv2(g, h)
        h = F.softmax(h, dim=1)  # Apply softmax over the classes dimension
        # h = F.sigmoid(h)

        return h

trained_net, bestLost, epoch, inp, lossList = train1('_80wayCut_Lossinter_min_cut_loss_9.pth')

Epoch: 0, Cumulative Loss: 2634424.9970703125
Epoch: 100, Cumulative Loss: 2528362.158203125
Epoch: 200, Cumulative Loss: 2528362.158203125
Epoch: 300, Cumulative Loss: 2528362.158203125
Epoch: 400, Cumulative Loss: 2528362.158203125
GNN training took 426.469 seconds.
Best cumulative loss: 10473.2529296875


In [58]:
def partition_loss(g, node_logits, terminals):
    # Convert logits to probabilities
    probabilities = torch.softmax(node_logits, dim=1)

    # Terminal node embeddings
    h_a, h_b, h_c = probabilities[terminals[0]], probabilities[terminals[1]], probabilities[terminals[2]]

    # Separation loss
    sep_loss = (torch.max(torch.tensor(0.0), 1 - torch.norm(h_a - h_b)) +
                torch.max(torch.tensor(0.0), 1 - torch.norm(h_b - h_c)) +
                torch.max(torch.tensor(0.0), 1 - torch.norm(h_c - h_a)))

    # Cut size minimization loss
    cut_loss = 0
    assignments = torch.argmax(probabilities, dim=1)
    for u, v in zip(*g.edges()):
        if assignments[u] != assignments[v]:
            cut_loss += 1  # Assuming uniform weight of 1 for each edge

    return sep_loss + cut_loss


def loss_terminal(s, g,  A=0, C=1, penalty=10000):
    loss = partition_loss(g, s, [0,1,2])
    # loss += (penalty* terminal_independence_penalty(s, [0,1,2]))
    return loss

class GCNSoftmax(nn.Module):
    # def __init__(self, in_feats, hidden_size, num_classes, dropout, device):
    #     super(GCNSoftmax, self).__init__()
    #     self.dropout_frac = dropout
    #     self.conv1 = GraphConv(in_feats, hidden_size).to(device)
    #     self.conv2 = GraphConv(hidden_size, num_classes).to(device)
    #
    # def forward(self, g, inputs):
    #     # Basic forward pass
    #     h = self.conv1(g, inputs)
    #     h = F.relu(h)
    #     h = F.dropout(h, p=self.dropout_frac, training=self.training)
    #     h = self.conv2(g, h)
    #     h = F.softmax(h, dim=1)  # Apply softmax over the classes dimension
    #     # h = F.sigmoid(h)
    #
    #     return h

    def __init__(self, in_feats, hidden_size, num_classes,dropout, device):
        super(GCNSoftmax, self).__init__()
        self.conv1 = dglnn.GraphConv(in_feats, hidden_size)
        self.conv2 = dglnn.GraphConv(hidden_size, hidden_size)
        self.classifier = nn.Linear(hidden_size, num_classes)

    def forward(self, g, features):
        # Apply graph convolution and activation layer
        h = torch.relu(self.conv1(g, features))
        h = torch.relu(self.conv2(g, h))
        # Classifier applied directly to node features
        return self.classifier(h)

trained_net, bestLost, epoch, inp, lossList = train1('_80wayCut_Lossinter_min_cut_loss_9.pth')

Epoch: 0, Cumulative Loss: 24380.278964996338
Epoch: 100, Cumulative Loss: 26443.733406066895
Epoch: 200, Cumulative Loss: 26389.7333984375
Epoch: 300, Cumulative Loss: 26405.733375549316
Epoch: 400, Cumulative Loss: 26403.733375549316
GNN training took 416.921 seconds.
Best cumulative loss: 40.904293060302734


AttributeError: 'Tensor' object has no attribute 'edges'

In [40]:
import torch
import torch.nn.functional as F

def probabilistic_cut_loss(s, adjacency_matrix):
    """
    Calculate the expected cut loss based on partition probabilities.

    :param s: A matrix of size [num_nodes, num_partitions] where each element is the probability of a node being in a partition.
    :param adjacency_matrix: The adjacency matrix of the graph.
    :return: The expected cut loss.
    """
    num_partitions = s.shape[1]
    cut_loss = 0
    for i in range(num_partitions):
        for j in range(i + 1, num_partitions):
            partition_i = s[:, i].unsqueeze(1)
            partition_j = s[:, j].unsqueeze(0)
            # Calculate the probability that nodes are in different partitions
            inter_partition_prob = partition_i * partition_j
            # Calculate the expected cut weight
            cut_loss += torch.sum(adjacency_matrix * inter_partition_prob)
    return cut_loss

def terminal_independence_penalty(s, terminals):
    """
    Enforce that terminal nodes are in separate partitions by penalizing overlaps.

    :param s: Probability matrix of node partition assignments.
    :param terminals: List of indices of terminal nodes.
    :return: Penalty for terminal nodes sharing partitions.
    """
    penalty = 0
    num_terminals = len(terminals)
    for i in range(num_terminals):
        for j in range(i + 1, num_terminals):
            # Calculate the dot product to determine the overlap in partition probabilities
            overlap = torch.dot(s[terminals[i]], s[terminals[j]])
            penalty += overlap  # Penalize any overlap
    return penalty

def loss_terminal(s, adjacency_matrix,  A= 0, C=1, T=100):
    """
    Compute the overall loss including cut loss and terminal independence.

    :param s: Node partition probabilities.
    :param adjacency_matrix: Graph adjacency matrix.
    :param terminals: List of terminal node indices.
    :param C: Weight for the cut loss.
    :param T: Weight for the terminal independence penalty.
    :return: Total loss.
    """
    cut_loss = probabilistic_cut_loss(s, adjacency_matrix)
    terminal_loss = terminal_independence_penalty(s, [0,1,2])
    total_loss = C * cut_loss + T * terminal_loss
    return total_loss


trained_net, bestLost, epoch, inp, lossList = train1('_80wayCut_Lossinter_min_cut_loss_9.pth')

Epoch: 0, Cumulative Loss: 111788.00930786133
Epoch: 100, Cumulative Loss: 75000.0
Epoch: 200, Cumulative Loss: 75000.0
Epoch: 300, Cumulative Loss: 75000.0
Epoch: 400, Cumulative Loss: 75000.0
GNN training took 155.964 seconds.
Best cumulative loss: 300.0


In [ ]:
trained_net, bestLost, epoch, inp, lossList = train1()

In [ ]:
trained_net_2, bestLost_2, epoch_2, inp_2, lossList_2 = train1()


### Testing on exp1

In [ ]:
# n, d, p, graph_type, number_epochs, learning_rate, PROB_THRESHOLD, tol, patience, dim_embedding, hidden_dim = hyperParameters(n=30,patience=1000)
#
# # Establish pytorch GNN + optimizer
# opt_params = {'lr': learning_rate}
# gnn_hypers = {
#     'dim_embedding': dim_embedding,
#     'hidden_dim': hidden_dim,
#     'dropout': 0.0,
#     'number_classes': 3,
#     'prob_threshold': PROB_THRESHOLD,
#     'number_epochs': number_epochs,
#     'tolerance': tol,
#     'patience': patience,
#     'nodes':n
# }
#
# net, embed, optimizer = get_gnn(n, gnn_hypers, opt_params, TORCH_DEVICE, TORCH_DTYPE)
# # load nerual network model for evaluation
# model, inputs =LoadNeuralModel(net, gnn_hypers, TORCH_DEVICE, './exp1.pth')
# model.eval()
#
# #create dummy graph
# graph = CreateGraph(20)
# dgl_graph = dgl.from_networkx(nx_graph=graph)
# dgl_graph = dgl_graph.to(TORCH_DEVICE)
# q_torch = qubo_dict_to_torch(graph, gen_adj_matrix(graph), torch_dtype=TORCH_DTYPE, torch_device=TORCH_DEVICE)
#
# # find min cut
# print("Heurestic 3-way min-cut value: " + str(find3WayCut(graph, [0,16,19])))
# print("Neural Network 3-way min-cut value: " + str(GetOptimalNetValue(model,dgl_graph, inputs, q_torch, {0:0, 16:1, 19:2})))


In [ ]:
# test_item = {}
# for i in range(10):
#
#
#     graph = CreateGraph(30)
#     graph_dgl = dgl.from_networkx(nx_graph=graph)
#     graph_dgl = graph_dgl.to(TORCH_DEVICE)
#     q_torch = qubo_dict_to_torch(graph, gen_adj_matrix(graph), torch_dtype=TORCH_DTYPE, torch_device=TORCH_DEVICE)
#
#     test_item[i] = [graph_dgl, q_torch, graph]
#
# for key, (dgl_graph, adjacency_matrix,graph) in test_item.items():
#     print("Heurestic 3-way min-cut value: " + str(find3WayCut(graph, [0,20,28])))
#     print("Neural Network 3-way min-cut value: " + str(GetOptimalNetValue(model,dgl_graph, inputs, adjacency_matrix, {0:0, 20:1, 28:2})))
#     print(f'-------')


## Experiment 2

Creating training set with the following constraints:
- Each graph have precisey 100 nodes
- Each graph has random edges (50% probability of edge creation)
- Each edge has random value of 1-500

In [ ]:
# datasetItem = {}
# for i in range(1000):
#
#
#     graph = CreateGraph(100)
#     graph_dgl = dgl.from_networkx(nx_graph=graph)
#     graph_dgl = graph_dgl.to(TORCH_DEVICE)
#     q_torch = qubo_dict_to_torch(graph, gen_adj_matrix(graph), torch_dtype=TORCH_DTYPE, torch_device=TORCH_DEVICE)
#
#     datasetItem[i] = [graph_dgl, q_torch, graph]
#     print("Graph Number Created: "+str(i))

In [ ]:
# print(datasetItem[1][2].nodes)
# # Visualize graph
# pos = nx.kamada_kawai_layout(datasetItem[1][2])
# nx.draw(datasetItem[1][2], pos, with_labels=True, node_color=[[.7, .7, .7]])

In [ ]:
# n, d, p, graph_type, number_epochs, learning_rate, PROB_THRESHOLD, tol, patience, dim_embedding, hidden_dim = hyperParameters(n=100,patience=1000)
#
# # Establish pytorch GNN + optimizer
# opt_params = {'lr': learning_rate}
# gnn_hypers = {
#     'dim_embedding': dim_embedding,
#     'hidden_dim': hidden_dim,
#     'dropout': 0.0,
#     'number_classes': 3,
#     'prob_threshold': PROB_THRESHOLD,
#     'number_epochs': number_epochs,
#     'tolerance': tol,
#     'patience': patience,
#     'nodes':n
# }
#
# net, embed, optimizer = get_gnn(n, gnn_hypers, opt_params, TORCH_DEVICE, TORCH_DTYPE)
#
# terminal_nodes = 3
#
# trained_net, bestLost, epoch, inp= run_gnn_training(
#     datasetItem, net, embed, optimizer, int(1e3),
#     gnn_hypers['tolerance'], gnn_hypers['patience'], Loss, terminal_nodes, 100, gnn_hypers['number_classes'], './exp2.pth')


## Testing Exp2

In [ ]:
# n, d, p, graph_type, number_epochs, learning_rate, PROB_THRESHOLD, tol, patience, dim_embedding, hidden_dim = hyperParameters(n=100,patience=1000)
#
# # Establish pytorch GNN + optimizer
# opt_params = {'lr': learning_rate}
# gnn_hypers = {
#     'dim_embedding': dim_embedding,
#     'hidden_dim': hidden_dim,
#     'dropout': 0.0,
#     'number_classes': 3,
#     'prob_threshold': PROB_THRESHOLD,
#     'number_epochs': number_epochs,
#     'tolerance': tol,
#     'patience': patience,
#     'nodes':n
# }
#
# net, embed, optimizer = get_gnn(n, gnn_hypers, opt_params, TORCH_DEVICE, TORCH_DTYPE)
# # load nerual network model for evaluation
# model, inputs =LoadNeuralModel(net, gnn_hypers, TORCH_DEVICE, './exp2.pth')
# model.eval()
#
# #create dummy graph
# graph = CreateGraph(100)
# dgl_graph = dgl.from_networkx(nx_graph=graph)
# dgl_graph = graph_dgl.to(TORCH_DEVICE)
# q_torch = qubo_dict_to_torch(graph, gen_adj_matrix(graph), torch_dtype=TORCH_DTYPE, torch_device=TORCH_DEVICE)
#
# # find min cut
# print("Heurestic 3-way min-cut value: " + str(find3WayCut(graph, [0,20,28])))
# print("Neural Network 3-way min-cut value: " + str(GetOptimalNetValue(model,dgl_graph, inputs, q_torch, {0:0, 20:1, 28:2})))


In [ ]:
# test_item = {}
# for i in range(100):
#
#
#     graph = CreateGraph(100)
#     graph_dgl = dgl.from_networkx(nx_graph=graph)
#     graph_dgl = graph_dgl.to(TORCH_DEVICE)
#     q_torch = qubo_dict_to_torch(graph, gen_adj_matrix(graph), torch_dtype=TORCH_DTYPE, torch_device=TORCH_DEVICE)
#
#     test_item[i] = [graph_dgl, q_torch, graph]
#
# indices = []
# i = 0
# heurestic_cut = []
# neural_cut = []
# for key, (dgl_graph, adjacency_matrix,graph) in test_item.items():
#     heurestic_cut.append(find3WayCut(graph, [0,20,65]))
#     neural_cut.append(GetOptimalNetValue(model,dgl_graph, inputs, adjacency_matrix, {0:0, 20:1, 65:2}))
#     i+=1
#     indices.append(i)
#     print("Heurestic 3-way min-cut value: " + str(heurestic_cut[-1]))
#     print("Neural Network 3-way min-cut value: " + str(neural_cut[-1]))
#     print(f'-------')

In [ ]:
# # Function to plot True vs Prediction graph
# def barPlot(indices, heurestic_cut, neural_cut):
#     # Example data
#     n_groups = len(heurestic_cut)
#     index = np.arange(n_groups)
#     bar_width = 0.35
#
#     # Create bars
#     plt.figure(figsize=(30, 6))
#     bar1 = plt.bar(index, heurestic_cut, bar_width, label='Heurestic')
#     bar2 = plt.bar(index + bar_width, neural_cut, bar_width, label='Neural Network')
#
#     # Add details
#     plt.xlabel('Graph Number')
#     plt.ylabel('Minimum Cut Value')
#     plt.title('Comparison of Minimum Cut Values by Algorithm')
#     plt.xticks(index + bar_width / 2, range(1, n_groups + 1))
#     plt.legend()
#     plt.tight_layout()
#     plt.show()

In [ ]:
# barPlot(indices, heurestic_cut, neural_cut)

In [ ]:
# percentage = 0
# min_cut_value_less_than_heuristic = 0
# for i in indices:
#     if neural_cut[i-1] <= heurestic_cut[i-1]:
#         percentage+=1
#     if neural_cut[i-1] < heurestic_cut[i-1]:
#         min_cut_value_less_than_heuristic+=1
# print("Percentage better or equal min-cut value:" +  str(percentage))
# print("Percentage better min-cut value:" +  str(min_cut_value_less_than_heuristic))

In [ ]:
# test_item = {}
# for i in range(100):
#
#
#     graph = CreateGraph_random(100, 100)
#     graph_dgl = dgl.from_networkx(nx_graph=graph)
#     graph_dgl = graph_dgl.to(TORCH_DEVICE)
#     q_torch = qubo_dict_to_torch(graph, gen_adj_matrix(graph), torch_dtype=TORCH_DTYPE, torch_device=TORCH_DEVICE)
#
#     test_item[i] = [graph_dgl, q_torch, graph]
#
# indices = []
# i = 0
# heurestic_cut = []
# neural_cut = []
# for key, (dgl_graph, adjacency_matrix,graph) in test_item.items():
#     heurestic_cut.append(find3WayCut(graph, [0,20,65]))
#     neural_cut.append(GetOptimalNetValue(model,dgl_graph, inputs, adjacency_matrix, {0:0, 20:1, 65:2}))
#     i+=1
#     indices.append(i)
#     print("Heurestic 3-way min-cut value: " + str(heurestic_cut[-1]))
#     print("Neural Network 3-way min-cut value: " + str(neural_cut[-1]))
#     print(f'-------')

In [ ]:
# percentage = 0
# min_cut_value_less_than_heuristic = 0
# for i in indices:
#     if neural_cut[i-1] <= heurestic_cut[i-1]:
#         percentage+=1
#     if neural_cut[i-1] < heurestic_cut[i-1]:
#         min_cut_value_less_than_heuristic+=1
# print("Percentage better or equal min-cut value:" +  str(percentage))
# print("Percentage better min-cut value:" +  str(min_cut_value_less_than_heuristic))

## Experiment 3

Creating training set with the following constraints:
- Each graph have precisely 300 nodes
- Each graph has random edges (50% probability of edge creation)
- Each edge has random value of 1-500

In [ ]:
# datasetItem = {}
# for i in range(1000):
#
#
#     graph = CreateGraph_random(300, 500)
#     graph_dgl = dgl.from_networkx(nx_graph=graph)
#     graph_dgl = graph_dgl.to(TORCH_DEVICE)
#     q_torch = qubo_dict_to_torch(graph, gen_adj_matrix(graph), torch_dtype=TORCH_DTYPE, torch_device=TORCH_DEVICE)
#
#     datasetItem[i] = [graph_dgl, q_torch, graph]
#     print("Graph Number Created: "+str(i))

In [ ]:
# n, d, p, graph_type, number_epochs, learning_rate, PROB_THRESHOLD, tol, patience, dim_embedding, hidden_dim = hyperParameters(n=300,patience=1000)
#
# # Establish pytorch GNN + optimizer
# opt_params = {'lr': learning_rate}
# gnn_hypers = {
#     'dim_embedding': dim_embedding,
#     'hidden_dim': hidden_dim,
#     'dropout': 0.0,
#     'number_classes': 3,
#     'prob_threshold': PROB_THRESHOLD,
#     'number_epochs': number_epochs,
#     'tolerance': tol,
#     'patience': patience,
#     'nodes':n
# }
#
# net, embed, optimizer = get_gnn(n, gnn_hypers, opt_params, TORCH_DEVICE, TORCH_DTYPE)
#
# terminal_nodes = 3
#
# trained_net, bestLost, epoch, inp= run_gnn_training(
#     datasetItem, net, embed, optimizer, int(1e3),
#     gnn_hypers['tolerance'], gnn_hypers['patience'], Loss, terminal_nodes, 300, gnn_hypers['number_classes'], './exp3.pth')


In [ ]:
# n, d, p, graph_type, number_epochs, learning_rate, PROB_THRESHOLD, tol, patience, dim_embedding, hidden_dim = hyperParameters(n=300,patience=1000)
#
# # Establish pytorch GNN + optimizer
# opt_params = {'lr': learning_rate}
# gnn_hypers = {
#     'dim_embedding': dim_embedding,
#     'hidden_dim': hidden_dim,
#     'dropout': 0.0,
#     'number_classes': 3,
#     'prob_threshold': PROB_THRESHOLD,
#     'number_epochs': number_epochs,
#     'tolerance': tol,
#     'patience': patience,
#     'nodes':n
# }
#
# net, embed, optimizer = get_gnn(n, gnn_hypers, opt_params, TORCH_DEVICE, TORCH_DTYPE)
# # load nerual network model for evaluation
# model, inputs =LoadNeuralModel(net, gnn_hypers, TORCH_DEVICE, './exp3.pth')
# model.eval()
#


In [ ]:
'''
test_item = {}
for i in range(100):


    graph = CreateGraph_random(300, 500)
    graph_dgl = dgl.from_networkx(nx_graph=graph)
    graph_dgl = graph_dgl.to(TORCH_DEVICE)
    q_torch = qubo_dict_to_torch(graph, gen_adj_matrix(graph), torch_dtype=TORCH_DTYPE, torch_device=TORCH_DEVICE)

    test_item[i] = [graph_dgl, q_torch, graph]

indices = []
i = 0
heurestic_cut = []
neural_cut = []
for key, (dgl_graph, adjacency_matrix,graph) in test_item.items():
    heurestic_cut.append(find3WayCut(graph, [0,150,250]))
    neural_cut.append(GetOptimalNetValue(model,dgl_graph, inputs, adjacency_matrix, {0:0, 150:1, 250:2}))
    i+=1
    indices.append(i)
    print("Heurestic 3-way min-cut value: " + str(heurestic_cut[-1]))
    print("Neural Network 3-way min-cut value: " + str(neural_cut[-1]))
    print(f'-------')
'''

In [ ]:
# percentage = 0
# min_cut_value_less_than_heuristic = 0
# for i in indices:
#     if neural_cut[i-1] <= heurestic_cut[i-1]:
#         percentage+=1
#     if neural_cut[i-1] < heurestic_cut[i-1]:
#         min_cut_value_less_than_heuristic+=1
# print("Percentage better or equal min-cut value:" +  str(percentage))
# print("Percentage better min-cut value:" +  str(min_cut_value_less_than_heuristic))

In [ ]:
# from matplotlib.ticker import ScalarFormatter
# def barPlot_2(heurestic_cut, neural_cut):
#     # Example data
#     n_groups = len(heurestic_cut)
#     index = np.arange(n_groups)
#     bar_width = 0.35
#
#     # Create bars
#     plt.figure(figsize=(30, 6))
#     bar1 = plt.bar(index, heurestic_cut, bar_width, label='Heurestic', log=True)
#     bar2 = plt.bar(index + bar_width, neural_cut, bar_width, label='Neural Network', log=True)
#
#     # Add details
#     plt.xlabel('Graph Number')
#     plt.ylabel('Minimum Cut Value')
#     plt.title('Comparison of Minimum Cut Values by Algorithm')
#     plt.xticks(index + bar_width / 2, range(1, n_groups + 1))
#     plt.legend()
#     plt.tight_layout()
#     plt.gca().yaxis.set(major_formatter=ScalarFormatter(), minor_formatter=ScalarFormatter());
#     plt.show()

In [ ]:
# barPlot_2( heurestic_cut, neural_cut)